In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from src.subtitle.SubtitleToStorage import SubtitlesToStorage
import pandas as pd
from tqdm import tqdm
import torch

In [2]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')

Maybe helpful if you use CUDA 11.3
```shell
pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
```
More versions: https://pytorch.org/get-started/previous-versions/

In [6]:
device = 'cpu'
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)
model = model.to(device)

cuda:0


Change the source videos here:
- tom_scott_videos_all.csv
- two_minute_papers_all.csv

In [4]:
videos = pd.read_csv('../data/videos/tom_scott_videos_all.csv')
videos

,video_id,title
0,mwx_rumXUAw,Keeping the world's longest railroad tunnel safe
1,yWYkoZKHLfg,Why do YouTubers clap at the start of videos?
2,FJIzV0yYu0c,This massive truck makes artificial earthquakes
3,XxCha4Kez9c,How the US Postal Service reads terrible handw...
4,uAdmzyKagvE,"This clock was famous, but the internet ruined..."
...,...,...
360,0Sz55gmNUaI,The World's Largest Indoor Waterpark
361,ch7HwhGynXk,The Islands Where Guns are Required
362,eFJ-ze2-SqU,How To Visit Svalbard
363,oZ2Qms2fMH8,The Sundial That Works 24 Hours A Day


In [7]:
summaries = list()
for _, row in tqdm(videos.iterrows(), total=videos.shape[0]):
    subtitle_downloader = SubtitlesToStorage(
        source='youtube',
        video_id=row['video_id'],
        storage_config={'type': 'return_value'}
    )

    subtitles = subtitle_downloader.save()
    subtitles = subtitles.replace('\n', ' ')
    subtitles = subtitles.replace('- ', '')
    batch = tokenizer(subtitles, truncation=True, max_length=1024, return_tensors='pt')
    generated_ids = model.generate(batch["input_ids"].to(device), min_length=256, max_length=512)
    summary = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    summaries.append({'title': row['title'],  'summary': summary[0]})

  0%|          | 0/365 [00:04<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 2.00 GiB total capacity; 1.70 GiB already allocated; 0 bytes free; 1.72 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [5]:
summaries[0]

{'title': "NVIDIA's Image Restoration AI: Almost Perfect!",
 'summary': "Neural network-based solutions are amazing at creating clear, noise-free images. With light transport simulations, creating a noisy image means following the path of millions and millions of light rays, which can take up to hours per training sample. A collaboration between NVIDIA, AaltoUniversity and MIT, scientists came up with an insane idea: let's try to train a neural network without clear images and use only noisy data. This can help us restore images with significantoutlier content. The thought of this boggles my mind so much it keeps me up at night. I hope there will soon be followup papers that extend this idea to other problems as well. If you enjoyed this episode, please consider supporting us on patreon.com/TwoMinutePapers and there is also a link to it in the video. Thanks for watching and for your generous generoussupport, and I'll see you next time! Károly Zsolnai-Fehér, co-host of Two Minute Papers

In [23]:
df = pd.DataFrame(summaries)
df.to_csv('title_summary.csv', index=True, index_label='id', header=True)

In [24]:
test = pd.read_csv('title_summary.csv')
test

,id,title,summary
0,0,Reopening an airport terminal is harder than y...,The South Terminal at London's Gatwick Airport...
1,1,14 science fiction stories in under 6 minutes,I used to make short science fictionvideos for...
2,2,"After 140 years, this old technology still kee...","140 years ago, the Callander and Oban railway ..."
3,3,"The Elie Chainwalk is safe, as long as you fol...","The Elie Chainwalk is 500 metres long, made up..."
4,4,This town forgot to be a city,Rochester had been a city since the 13th centu...
...,...,...,...
236,236,Why California's Musical Road Sounds Terrible,If you drive down a certain stretch of highway...
237,237,The Reaction Ferries of Basel: What Have We Mi...,"CNN's John Sutter travels to Basel, Switzerlan..."
238,238,The Centuries-Old Debt That's Still Paying Int...,"In 1648, the Water Board of Lekdijk Bovendams ..."
239,239,Rotary Jails and Accidental Amputations,In 1881 an architect called William Brown and ...
